# Agent Lab – Clean Room Notebook

This notebook mirrors the CLI experience documented in the repository and keeps the
logic in sync with the reusable `agent_lab` package. Use it to experiment with the
Loan Risk Assistant without duplicating boilerplate configuration.

## 1. Imports

The notebook relies exclusively on the `agent_lab` module so any updates to the
package (prompt tweaks, new tools, authentication fixes, etc.) are picked up here
without manual copy/paste.

In [ ]:

from __future__ import annotations

import os
from typing import Any, Iterable

from langchain_core.messages import AIMessage, HumanMessage

from agent_lab import (
    AgentContext,
    DEFAULT_INSTRUCTIONS,
    ModelConfig,
    ModelParameters,
    RagToolConfig,
    Workspace,
    build_agent,
    load_credentials,
)


## 2. Helper utilities

A couple of helpers keep the notebook tidy:

* `resolve_workspace` mirrors the CLI logic so project/space IDs can come from either
  the environment or ad-hoc overrides.
* `convert_messages` adapts raw role/content dictionaries into LangChain objects for
  seamless hand-off to the agent graph.

In [ ]:

def resolve_workspace(project_id: str | None = None, space_id: str | None = None) -> Workspace:
    """Load watsonx workspace identifiers with validation."""

    env_project_id = project_id or os.getenv("PROJECT_ID") or os.getenv("WATSONX_PROJECT_ID")
    env_space_id = space_id or os.getenv("SPACE_ID") or os.getenv("WATSONX_SPACE_ID")

    if not env_project_id and not env_space_id:
        raise RuntimeError(
            "A watsonx project or space ID is required. Set PROJECT_ID/SPACE_ID (or their WATSONX_* variants)."
        )

    return Workspace(project_id=env_project_id, space_id=env_space_id)


def convert_messages(messages: Iterable[dict[str, Any]]):
    converted_messages: list[HumanMessage | AIMessage] = []
    for message in messages:
        if message["role"] == "user":
            converted_messages.append(HumanMessage(content=message["content"]))
        elif message["role"] == "assistant":
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages


## 3. Build the agent

Credentials will be pulled from the environment (with an interactive fallback if the
`IBM_API_KEY` variable is missing). Update the `vector_index_id` or model parameters to
point at your own artifacts.

In [ ]:

credentials, source = load_credentials()
workspace = resolve_workspace()

rag_config = RagToolConfig(
    vector_index_id=os.getenv("VECTOR_INDEX_ID", "40824957-150a-4607-a08c-7f8885b0befa"),
)

model_config = ModelConfig(
    model_id=os.getenv("GRANITE_MODEL", "ibm/granite-3-3-8b-instruct"),
    parameters=ModelParameters(max_tokens=1500, temperature=0.1),
)

agent_context = AgentContext(
    credentials=credentials,
    workspace=workspace,
    model=model_config,
    instructions=DEFAULT_INSTRUCTIONS,
    rag=rag_config,
)

agent = build_agent(agent_context)

print("Agent ready. Credential source ->", source)


## 4. Ask questions

`run_notebook_prompt` wraps the invocation boilerplate so you can focus on the prompt
engineering workflow.

In [ ]:

def run_notebook_prompt(prompt: str, *, thread_id: str = "notebook-demo"):
    messages = [{"role": "user", "content": prompt}]
    response = agent.invoke({"messages": convert_messages(messages)}, {"configurable": {"thread_id": thread_id}})
    return response["messages"][-1].content


### Example query

Feel free to edit the YAML payload or replace it with a free-form question.

In [ ]:

sample_prompt = """high_risk:
  applicant_id: 1001
  risk_score: 0.85
  credit_score: 580
  debt_to_income: 0.62
  explanation: "Low credit score, high DTI"
"""

print(run_notebook_prompt(sample_prompt))
